#### Flux with PCA

##### Libraries

In [ ]:
using MLDatasets           # mnist
using Images
using PreprocessingImages; pim = PreprocessingImages
using PreprocessingArrays; pa  = PreprocessingArrays

using MLJ                  # make_blobs, rmse, confmat, f1score, coerce
using MLDataUtils          # label, nlabel, labelfreq, stratifiedobs
using Flux                 # the julia ml library

using Random
using DataFrames

In [ ]:
include("../libs/libMisc.jl")

##### MNIST

In [ ]:
# load mnist
datasetX,    datasetY    = MNIST(:train)[:]
validationX, validationY = MNIST(:test)[:]

display( size(datasetX) )

img  = datasetX[:, :, 1:5]
img2 = permutedims(img, (2, 1, 3))

display(datasetY[1:5]')
mosaicview( Gray.(img2)  ; nrow=1)

In [ ]:
# split trainset, testset from dataset
Random.seed!(1)
(trainX, trainY), (testX, testY) = stratifiedobs((datasetX, datasetY), p = 0.7)
size(trainX), size(testX), size(validationX)

### Preprocessing


In [ ]:
function preprocess(X, y)
    Xs = pim.batchImage2DF(X)
    ys = Int32.(y)

    return (Xs, ys)
end

X_tr, y_tr = preprocess(trainX, trainY);

In [ ]:
# reduce predictors
PCA = @load PCA pkg=MultivariateStats verbosity=0
reducer = PCA(pratio = 0.95)

# standardize predictors
std = Standardizer()

# execute
pipe = @pipeline reducer std
mach = MLJ.machine(pipe, X_tr) |> fit!
X_tr = MLJ.transform(mach, X_tr) .|> Float32;   # transform(unsupervised) vs predict(supervised)

In [ ]:
function preprocess2(X, y)
    N, d = size(X)
    Xs = X |> Matrix 
    Xs = Float32.(Xs) |> Flux.flatten
    Xs = [Xs[i,:] for i in 1:N]
    Xs = Flux.batch(Xs)
    
    ys = Flux.onehotbatch( Float32.(y), 0:9 )
    
    return (Xs, ys)
end

X_tr2, y_tr2 = preprocess2(X_tr, y_tr);

##### Model

In [ ]:
# model configuration
nInputs  = size(X_tr2)[1]
nOutputs = 10

model              = Chain( Flux.Dense(nInputs, nOutputs, tanh),   # tanh is chosen as nonlinearity (Prof Mostafa lecture)
                            softmax )                              # softmax scales the output to sum to one
lossFunction(X, y) = Flux.mse( model(X), y )
modelParameters    = Flux.params(model)
data               = Flux.DataLoader((X_tr2, y_tr2), batchsize=1)  # default batchsize=1
modelOptimizer     = Flux.Descent()
callBack           = Flux.throttle(() -> println("training"), 10); # print every 10s

##### Training

In [ ]:
numberOfEpochs = 10

In [ ]:
# preferred for multiple epochs
epochs = 1:numberOfEpochs
for epoch in epochs
    Flux.train!(lossFunction, modelParameters, data, modelOptimizer; cb=callBack)
end

##### Testing

In [ ]:
function predictOutcome(X)
    ŷ = Flux.onecold( model(X), [0:9;] )
end

In [ ]:
# preprocessing
X_ts, y_ts = preprocess(testX, testY)
X_ts       = MLJ.transform(mach, X_ts)
X_ts, y_ts = preprocess2(X_ts, y_ts)

# predict
ŷ = predictOutcome(X_ts);

In [ ]:
printMetrics( ŷ, coerce(testY, OrderedFactor) )